#### Import packages

In [1]:
# import torch.multiprocessing as mp
# mp.set_start_method('spawn') 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import shutil
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import cv2

from keras.models import Sequential, Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation
from keras.layers import UpSampling2D, BatchNormalization, Flatten, Dense, GlobalMaxPooling2D
from keras.layers.core import Dropout, Reshape
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.optimizers import RMSprop, Adam

seed = 0
np.random.seed(seed)
from utils import *

Using TensorFlow backend.
/home/arsh/anaconda3/envs/deepLearn/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Load and Prepare Data

In [2]:
data = pd.read_json('./Data/train.json')
test = pd.read_json('./Data/test.json')

In [3]:
data['band_1'] = data['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
# mean_band1 = data['band_1'].mean().mean()
# sd_band1 = np.array(data['band_1']).std()
# data['band_1'] = (data['band_1'] - mean_band1)/sd_band1
data['band_2'] = data['band_2'].apply(lambda x : np.array(x).reshape(75, 75))
# mean_band2 = data['band_2'].mean().mean()
# sd_band2 = np.array(data['band_2']).std()
# data['band_2'] = (data['band_2'] - mean_band2)/sd_band2

test['band_1'] = test['band_1'].apply(lambda x : np.array(x).reshape(75, 75))
# test['band_1'] = (test['band_1'] - mean_band1)/sd_band1
test['band_2'] = test['band_2'].apply(lambda x : np.array(x).reshape(75, 75))
# test['band_2'] = (test['band_2'] - mean_band2)/sd_band2

data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce').fillna(0.0)
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')

In [4]:
ids_train = np.array(data.index)
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

#### Keras Model

In [6]:
# Hyper Parameters
width = 75
height = 75
channels = 2

In [7]:
def train_generator():
    while True:
        for start in range(0, len(ids_train_split), batch_size):
            x_batch_0 = []
            x_batch_1 = []
            y_batch = []
            end = min(start + batch_size, len(ids_train_split))
            ids_train_batch = ids_train_split[start:end]
            for idx in ids_train_batch:
                
                train = data.iloc[idx,]
                band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, 75, 75)
                band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, 75, 75)
                X_train = np.concatenate((band_1_tr, band_2_tr), axis=0)
                X_train = [X_train, np.array(train['inc_angle'])]

                im = X_train[0]
                im = randomErodeDilate(im, u=0.2)
                im = randomZoomOut(im, u=0.5)
                im = randomNoisy(im, u=0.4)
                im = randomShift(im, u=0.3)
                x_batch_0.append(im.reshape((75,75,2)))
                x_batch_1.append(X_train[1].astype(np.float32))
                y_batch.append(train['is_iceberg'].astype(np.float32))
            
            x_batch = [np.array(x_batch_0, np.float32), np.array(x_batch_1, np.float32)]
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch


def valid_generator():
    while True:
        for start in range(0, len(ids_valid_split), batch_size):
            x_batch_0 = []
            x_batch_1 = []
            y_batch = []
            end = min(start + batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for idx in ids_valid_batch:
                
                val = data.iloc[idx,]
                band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, 75, 75)
                band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, 75, 75)
                X_val = np.concatenate((band_1_val, band_2_val), axis=0)
                X_val = [X_val, np.array(val['inc_angle'])]

                im = X_val[0]
                x_batch_0.append(im.reshape((75,75,2)))
                x_batch_1.append(X_val[1].astype(np.float32))
                y_batch.append(val['is_iceberg'].astype(np.float32))
            
            x_batch = [np.array(x_batch_0, np.float32), np.array(x_batch_1, np.float32)]
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch

In [12]:
def get_model():
    input_1 = Input(shape=(width, height, channels), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    out = BatchNormalization()(input_1)
    # Layers 1+2
    out = Conv2D(9, (3, 3), padding='valid')(input_1)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(18, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
    out = Dropout(0.2)(out)
    
    # Layers 2+3
    out = Conv2D(24, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(36, (3, 3), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
    out = Dropout(0.2)(out)
    
    # Layer 4
    out = Conv2D(72, (2, 2), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
    out = Dropout(0.2)(out)
    
    # Layer 5
    out = Conv2D(144, (2, 2), padding='valid')(out)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = MaxPooling2D((2, 2), strides=(2, 2))(out)
    out = Dropout(0.2)(out)
    
    out = Flatten()(out)
    out_concat =  (Concatenate()([out, BatchNormalization()(input_2)]))
    
    # FCs 1+2+3
    out = Dense(512, activation='relu')(out_concat)
    out = Dropout(0.5)(out)
    out = Dense(196, activation='relu')(out)
    out = Dropout(0.3)(out)
    out = Dense(1, activation="sigmoid")(out)

    model = Model([input_1,input_2],  out)
    optimizer = Adam(lr=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()

In [ ]:
epochs = 300
batch_size = 32

callbacks = [EarlyStopping(monitor='val_loss',
                           patience=100,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=60,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='Keras_models/best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='keras_logs')]

model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Epoch 1/300
41/41 [==============================] - 1s - loss: 0.9791 - acc: 0.5395 - val_loss: 0.6567 - val_acc: 0.5483
Epoch 2/300
41/41 [==============================] - 1s - loss: 0.7181 - acc: 0.5677 - val_loss: 0.6594 - val_acc: 0.4984
Epoch 3/300
41/41 [==============================] - 1s - loss: 0.6401 - acc: 0.6057 - val_loss: 0.6316 - val_acc: 0.7009
Epoch 4/300
41/41 [==============================] - 1s - loss: 0.6152 - acc: 0.6161 - val_loss: 0.6010 - val_acc: 0.7259
Epoch 5/300
41/41 [==============================] - 1s - loss: 0.5838 - acc: 0.6642 - val_loss: 0.5295 - val_acc: 0.7477
Epoch 6/300
41/41 [==============================] - 1s - loss: 0.5613 - acc: 0.6782 - val_loss: 0.5273 - val_acc: 0.7072
Epoch 7/300
41/41 [==============================] - 1s - loss: 0.5499 - acc: 0.6912 - val_loss: 0.5879 - val_acc: 0.6199
Epoch 8/300
41/41 [==============================] - 1s - loss: 0.5195 - acc: 0.7209 - val_loss: 0.4668 - val_acc: 0.7913
Epoch 9/300
41/41 [=====

In [126]:
sum(y_train>0.5)/len(y_train)

0.45908028059236167

In [130]:
sum(np.array(results)>0.5)/len(results)

0.30282526115859448

In [132]:
sub = pd.read_csv('./Data/sample_submission.csv')

In [133]:
sub['is_iceberg'] = results

In [135]:
sub.to_csv('./Submissions/sub_30Oct_val_1631.csv', index=False)

#### KFold

In [55]:
band_1_KF = np.concatenate([im for im in data['band_1']]).reshape(-1, 75, 75)
band_2_KF = np.concatenate([im for im in data['band_2']]).reshape(-1, 75, 75)
rgb = np.stack((band_1_KF, band_2_KF), axis=1)
X_KF = [rgb, np.array(data['inc_angle']).reshape((len(data), 1))]

y_KF = data['is_iceberg'].values.astype(np.float32)

In [56]:
def accuracy(loader):
    iceNet.eval()
    correct = 0
    total = 0
    loss = []
    for features, features_angle, labels in loader:
        features = Variable(features, volatile=True).cuda()
        features_angle = Variable(features_angle, volatile=True).cuda()
        labels = Variable(labels, volatile=True).cuda()
        outputs = iceNet(features, features_angle)
        _loss = criterion(outputs, labels)
        loss.append(_loss)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.data).sum()
        
    return np.mean(loss).data[0], (100 * correct / total)

In [57]:
num_epochs = 300
batch_size = 32
learning_rate = 0.001

In [60]:
kfold = 5
kfold_scores = []

test_dataset = icebergDataset(X_test)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

results = []
sss = KFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(sss.split(X_KF[0], X_KF[1], y_KF)):
    X_train_KF, X_valid_KF = [X_KF[0][train_index], X_KF[1][train_index]], [X_KF[0][test_index], X_KF[1][test_index]]
    y_train_KF, y_valid_KF = y_KF[train_index], y_KF[test_index]
    
    # Define model
    iceNet = net().cuda()
    
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(iceNet.parameters(), lr=learning_rate)
    
    # Data Loader
    train_dataset_KF = icebergDataset(X_train_KF, y_train_KF, transform=True)
    val_dataset_KF = icebergDataset(X_valid_KF, y_valid_KF)

    train_loader_KF = torch.utils.data.DataLoader(dataset=train_dataset_KF, batch_size=batch_size, shuffle=True)
    val_loader_KF = torch.utils.data.DataLoader(dataset=val_dataset_KF, batch_size=batch_size, shuffle=False)
    
    print('Fold [%d/%d]' % (i+1, kfold))
    # Train
    best_prec1 = 1
    for epoch in range(num_epochs):
        epoch_train_loss = []
        for idx, (features, features_angle, labels) in enumerate(train_loader_KF):
            iceNet.train()
            features = Variable(features).cuda()
            features_angle = Variable(features_angle).cuda()
            labels = Variable(labels).cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = iceNet(features, features_angle)
            loss = criterion(outputs, labels)
            epoch_train_loss.append(loss)
            loss.backward()
            optimizer.step()

        prec1 = accuracy(val_loader_KF)[0]
        
        # Save best model
        is_best = prec1 < best_prec1
        best_prec1 = min(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': iceNet.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filename='./Models/v1Nov_v1/model_fold_'+str(i+1)+'.pth.tar')
        
    print('Val Score : %f' % (best_prec1))
    kfold_scores.append(best_prec1)
    # Load best model
    best_model = torch.load('./Models/v1Nov_v1/model_fold_'+str(i+1)+'.pth.tar')
    iceNet.load_state_dict(best_model['state_dict'])
    optimizer.load_state_dict(best_model['optimizer'])
    
    # Predict
    iceNet.eval()
    
    results_fold = []
    for features, features_angle in test_loader:
        iceNet.eval()
        features = Variable(features, volatile=True).cuda()
        features_angle = Variable(features_angle, volatile=True).cuda()
        outputs = F.softmax(iceNet(features, features_angle))
    #     outputs = iceNet(features, features_angle)

        results_fold.append(outputs.data[0][1])
    
    results.append(results_fold)

/home/arsh/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel/__main__.py:8: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Fold [1/5]
Val Score : 0.144305
Fold [2/5]
Val Score : 0.150032
Fold [3/5]
Val Score : 0.203240
Fold [4/5]
Val Score : 0.137823
Fold [5/5]
Val Score : 0.113948


In [61]:
np.array(kfold_scores).mean()

0.14986965507268907

In [52]:
sub = pd.read_csv('./Data/sample_submission.csv')
sub['is_iceberg'] = np.array(results).mean(axis=0)
sub.to_csv('./Submissions/Sub 4 - 10-fold _ Val-1269.csv', index=False)